In [ ]:
import os
import numpy as np
import cv2
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers,models
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import load_model
import cv2

from tensorflow import keras
from keras.layers import Input
from keras.models import Model
from keras.callbacks import Callback
import matplotlib.pyplot as plt

#from tensorflow.keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from skimage.segmentation import mark_boundaries
import keras.utils as image
from keras.utils import load_img,img_to_array,array_to_img
from matplotlib import cm

from collections import Counter
from google.colab import drive


In [ ]:

# Load pre-trained ResNet50 model
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom head
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs=base_model.input, outputs=x)

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Data preprocessing
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/chest_xray/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

test_data = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/chest_xray/chest_xray/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')


94765736/94765736 [==============================] - 0s 0us/step
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:

# Load the pre-trained ResNet50V2 model
base_model = keras.applications.ResNet50V2(weights='imagenet', include_top=False)

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model on top of the base model
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1, activation='relu')(x)
model = models.Model(inputs, x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load the pneumonia dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/chest_xray/train',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/chest_xray/chest_xray/val',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)



94668760/94668760 [==============================] - 1s 0us/step
Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=1)

163/163 [==============================] - 395s 2s/step - loss: 3.9205 - accuracy: 0.7429 - val_loss: 7.6246 - val_accuracy: 0.5000


In [ ]:
#model.summary()

In [ ]:
#model.save('/content/drive/MyDrive/Model/gradcam2.h5')

In [ ]:
# Load the pre-trained ResNet50V2 model
'''
input_tensor = layers.Input(shape=(224,224, 3))
base_model = keras.applications.ResNet50V2(weights='imagenet', include_top=True, input_tensor=input_tensor)

# Freeze all the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer
x = base_model.layers[-2].output
x = keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model
model = keras.models.Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load your dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/chest_xray/train',
    image_size=(224, 224),
    batch_size=32
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/chest_xray/chest_xray/val',
    image_size=(224, 224),
    batch_size=32
)
'''

# Load the pre-trained ResNet50V2 model
input_tensor = layers.Input(shape=(224,224, 3))
base_model = keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_tensor=input_tensor)
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1, activation='relu')(x)
model = models.Model(inputs, x)

# Freeze all the layers in the base model except the last t
t = 20
for layer in base_model.layers[:-t]:
    layer.trainable = False

# Add a new output layer
'''
x = base_model.layers[-2].output
x = keras.layers.Dense(1, activation='sigmoid')(x)
'''
# Create the new model
#model = keras.models.Model(inputs=base_model.input, outputs=x)

# Set up data augmentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

# Compile the model with a lower learning rate and binary crossentropy loss
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Load your dataset with data augmentation
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/chest_xray/train',
    image_size=(224, 224),
    batch_size=32
)
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/chest_xray/test',
    image_size=(224, 224),
    batch_size=32
)

# Train the model for the first few epochs with the base layers frozen
model.fit(train_dataset, epochs=1, validation_data=val_dataset)

# Unfreeze the last t layers
for layer in base_model.layers:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=keras.optimizers.Adam(lr=0.00001), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model for additional epochs
model.fit(train_dataset, epochs=1, validation_data=val_dataset)




Found 5216 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
163/163 [==============================] - 174s 1s/step - loss: 0.7812 - accuracy: 0.8602 - val_loss: 2.4407 - val_accuracy: 0.8029


163/163 [==============================] - 135s 599ms/step - loss: 0.7601 - accuracy: 0.8334 - val_loss: 5.7187 - val_accuracy: 0.6250


In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=1, validation_data=val_dataset)



163/163 [==============================] - 96s 577ms/step - loss: 0.4938 - accuracy: 0.8094 - val_loss: 3.1052 - val_accuracy: 0.6683


In [ ]:

# Unfreeze some layers
for layer in base_model.layers:
    layer.trainable = True

# Recompile the model
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:

# Fine-tune the model
history = model.fit(train_dataset, epochs=1, validation_data=val_dataset)

Epoch 1/10
163/163 [==============================] - 45s 218ms/step - loss: 0.1458 - accuracy: 0.9396 - val_loss: 0.2547 - val_accuracy: 0.9375
Epoch 2/10
163/163 [==============================] - 38s 224ms/step - loss: 0.0782 - accuracy: 0.9678 - val_loss: 0.2930 - val_accuracy: 0.8750
Epoch 3/10
163/163 [==============================] - 37s 217ms/step - loss: 0.0475 - accuracy: 0.9831 - val_loss: 0.6866 - val_accuracy: 0.8125
Epoch 4/10
163/163 [==============================] - 37s 217ms/step - loss: 0.0410 - accuracy: 0.9856 - val_loss: 1.1520 - val_accuracy: 0.6250
Epoch 5/10
163/163 [==============================] - 38s 224ms/step - loss: 0.0294 - accuracy: 0.9891 - val_loss: 1.5436 - val_accuracy: 0.5625
Epoch 6/10
163/163 [==============================] - 37s 215ms/step - loss: 0.0203 - accuracy: 0.9923 - val_loss: 1.0430 - val_accuracy: 0.6250
Epoch 7/10
163/163 [==============================] - 38s 223ms/step - loss: 0.0231 - accuracy: 0.9921 - val_loss: 0.6587 - val_ac

In [ ]:
#model.save_weights('/content/drive/MyDrive/Tumor2/my_model_new_weights.h5')

In [ ]:

# Load the pre-trained ResNet50V2 model
base_model = keras.applications.ResNet50V2(weights='imagenet', include_top=False)
#base_model=model
# Get the input tensor shape
input_tensor_shape = (224, 224, 3)
batch_size=32
# Freeze all the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add a new output layer
x = base_model.layers[-2].output
x = keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model
model = keras.models.Model(inputs=base_model.input, outputs=x)

# Load your dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/chest_xray/train',
    image_size=(input_tensor_shape[0], input_tensor_shape[1]),
    batch_size=batch_size
)


Found 5216 files belonging to 2 classes.


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
# Fine-tune the model
y_train = np.expand_dims(y_train, axis=-1)
y_val = np.expand_dims(y_val, axis=-1)
history = model.fit(train_dataset, epochs=1, validation_data=val_dataset)

NameError: ignored

In [ ]:

# Load the image and preprocess it
img_path = '/content/drive/MyDrive/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.resnet50.preprocess_input(x)

# Get the last convolutional layer
last_conv_layer = model.get_layer('conv5_block3_out')

# Compute the gradient of the class output value with respect to the feature map of the last convolutional layer
grad_model = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
with tf.GradientTape() as tape:
    preds, conv_outputs = grad_model(x)
    class_idx = tf.argmax(preds[0])
    loss = preds[:, class_idx]
grads = tape.gradient(loss, conv_outputs)[0]

# Compute the channel-wise mean of the gradients
weights = tf.reduce_mean(grads, axis=(0, 1))

# Multiply each channel in the feature map array by "how important this channel is" with regard to the class
cam = np.ones(conv_outputs.shape[1:3], dtype=np.float32)
for i, w in enumerate(weights):
    cam += w * conv_outputs[0, :, :, i]
# Normalize the heatmap between 0 and 255
heatmap = cam.numpy()
heatmap = cv2.resize(heatmap, (224,224))
heatmap = np.maximum(heatmap, 0)
heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap) + 1e-8)
heatmap = (heatmap * 255).astype(np.uint8)
# Apply a color map to the heatmap
jet = cm.get_cmap("jet")
heatmap = jet(heatmap)
heatmap = np.uint8(heatmap * 255)

# Convert the heatmap to RGB format
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGBA2RGB)

# Resize the heatmap to be the same size as the original image
heatmap = cv2.resize(heatmap, (224,224))

# Combine the heatmap with the original image
superimposed_img = cv2.addWeighted(np.array(img), 0.5, heatmap, 0.5, 0)

# Show the result
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()


InvalidArgumentError: ignored

In [ ]:


# Load an image to predict
#img_path = '/content/drive/MyDrive/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0

# Predict the class probabilities of the image
probs = model.predict(img_array)[0]

# Print the predicted class
if probs[0] > 0.5:
    print('Pneumonia detected with probability:', probs[0])
else:
    print('No Pneumonia detected with probability:', 1 - probs[0])


In [ ]:
#model.save('/content/drive/MyDrive/Tumor2/tumor_new_model.h5')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

# Set the conv3_block3_out layer to be trainable
base_model.get_layer('conv5_block3_out').trainable = True

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_dataset, epochs=100, validation_data=val_dataset)

In [ ]:

# Load the image and preprocess it
img_path = '/content/drive/MyDrive/chest_xray/val/PNEUMONIA/person1946_bacteria_4875.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = tf.keras.applications.resnet50.preprocess_input(x)

# Get the last convolutional layer
last_conv_layer = model.get_layer('conv5_block3_out')

# Compute the gradient of the class output value with respect to the feature map of the last convolutional layer
grad_model = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output])
with tf.GradientTape() as tape:
    preds, conv_outputs = grad_model(x)
    class_idx = tf.argmax(preds[0])
    loss = preds[:, class_idx]
grads = tape.gradient(loss, conv_outputs)[0]

# Compute the channel-wise mean of the gradients
weights = tf.reduce_mean(grads, axis=(0, 1))

# Multiply each channel in the feature map array by "how important this channel is" with regard to the class
cam = np.ones(conv_outputs.shape[1:3], dtype=np.float32)
for i, w in enumerate(weights):
    cam += w * conv_outputs[0, :, :, i]
# Normalize the heatmap between 0 and 255
heatmap = cam.numpy()
heatmap = cv2.resize(heatmap, (224,224))
heatmap = np.maximum(heatmap, 0)
heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap) + 1e-8)
heatmap = (heatmap * 255).astype(np.uint8)
# Apply a color map to the heatmap
jet = cm.get_cmap("jet")
heatmap = jet(heatmap)
heatmap = np.uint8(heatmap * 255)

# Convert the heatmap to RGB format
heatmap = cv2.cvtColor(heatmap, cv2.COLOR_RGBA2RGB)

# Resize the heatmap to be the same size as the original image
heatmap = cv2.resize(heatmap, (224,224))

# Combine the heatmap with the original image
superimposed_img = cv2.addWeighted(np.array(img), 0.5, heatmap, 0.5, 0)

# Show the result
plt.imshow(superimposed_img)
plt.axis('off')
plt.show()
